In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

data_dir = '[your path]'
log_dir = os.path.join(data_dir)

# Training

In [3]:
!pip install lightning torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 41.5 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings("ignore")

import json
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AutoModel, AutoTokenizer
from torchmetrics import CosineSimilarity
import torchmetrics
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
import os

# Set the seed for reproducibility
torch.manual_seed(42)

class TimeSeriesPolicyEmbeddingModel(pl.LightningModule):
    def __init__(self, model_name, time_series_hidden_size=128, sector_embedding_dim=16, embedding_dim=128, lr=1e-5):
        super(TimeSeriesPolicyEmbeddingModel, self).__init__()
        self.save_hyperparameters()

        # Time series + sector branch (simplified with MLP)
        self.time_series_proj = nn.Sequential(
            nn.Linear(448, time_series_hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(time_series_hidden_size, embedding_dim)
        )
        self.sector_proj = nn.Linear(22, sector_embedding_dim)
        self.final_proj = nn.Linear(embedding_dim + sector_embedding_dim, embedding_dim)

        # Text (summary) branch (tower) using BERT
        self.bert = AutoModel.from_pretrained(model_name)
        self.text_proj = nn.Linear(self.bert.config.hidden_size, embedding_dim)

        # Learning rate
        self.lr = lr

    def get_ts_sector_embedding(self, time_series, sector):
        time_series_embedding = self.time_series_proj(time_series)
        sector_embedding = self.sector_proj(sector).unsqueeze(1).expand(-1, time_series_embedding.size(1), -1)
        combined_ts_sector = torch.cat((time_series_embedding, sector_embedding), dim=-1)
        ts_sector_embedding = self.final_proj(combined_ts_sector.mean(dim=1))
        return ts_sector_embedding

    def get_text_embedding(self, input_ids, attention_mask):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        text_embedding = bert_outputs.pooler_output
        text_embedding = self.text_proj(text_embedding)
        return text_embedding

    def forward(self, time_series, sector, input_ids, attention_mask):
        ts_sector_embedding = self.get_ts_sector_embedding(time_series, sector)
        text_embedding = self.get_text_embedding(input_ids, attention_mask)
        return ts_sector_embedding, text_embedding

    def compute_triplet_loss(self, anchor_embedding, positive_embedding, negative_embedding, margin=1.0):
        positive_distance = torch.nn.functional.pairwise_distance(anchor_embedding, positive_embedding, p=2)
        negative_distance = torch.nn.functional.pairwise_distance(anchor_embedding, negative_embedding, p=2)
        loss = torch.relu(positive_distance - negative_distance + margin)
        return loss.mean()

    def training_step(self, batch, batch_idx):
        anchor_summary = batch['anchor_summary']
        positive_time_series = batch['positive_time_series']
        positive_sector = batch['positive_sector']
        negative_time_series = batch['negative_time_series']
        negative_sector = batch['negative_sector']

        anchor_embedding = self.get_text_embedding(anchor_summary['input_ids'], anchor_summary['attention_mask'])
        positive_embedding = self.get_ts_sector_embedding(positive_time_series, positive_sector)
        negative_embedding = self.get_ts_sector_embedding(negative_time_series, negative_sector)
        loss = self.compute_triplet_loss(anchor_embedding, positive_embedding, negative_embedding)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        anchor_summary = batch['anchor_summary']
        positive_time_series = batch['positive_time_series']
        positive_sector = batch['positive_sector']
        negative_time_series = batch['negative_time_series']
        negative_sector = batch['negative_sector']

        anchor_embedding = self.get_text_embedding(anchor_summary['input_ids'], anchor_summary['attention_mask'])
        positive_embedding = self.get_ts_sector_embedding(positive_time_series, positive_sector)
        negative_embedding = self.get_ts_sector_embedding(negative_time_series, negative_sector)
        loss = self.compute_triplet_loss(anchor_embedding, positive_embedding, negative_embedding)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}


# Custom Dataset for TimeSeriesPolicy with Triplet Data (anchor, positive, negative)
class TimeSeriesPolicyTripletDataset(Dataset):
    def __init__(self, json_file, tokenizer, max_length=512):
        with open(json_file, 'r') as file:
            self.data = [json.loads(line) for line in file]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        anchor_summary = self.tokenizer(
            self.data[idx]['anchor_summary'],
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.max_length
        )
        anchor_summary = {k: v.squeeze(0) for k, v in anchor_summary.items()}
        positive_time_series = torch.tensor(self.data[idx]['positive_time_series'], dtype=torch.float32)
        positive_sector = torch.tensor(self.data[idx]['positive_sector'], dtype=torch.float32)
        negative_time_series = torch.tensor(self.data[idx]['negative_time_series'], dtype=torch.float32)
        negative_sector = torch.tensor(self.data[idx]['negative_sector'], dtype=torch.float32)

        return {
            'anchor_summary': anchor_summary,
            'positive_time_series': positive_time_series,
            'positive_sector': positive_sector,
            'negative_time_series': negative_time_series,
            'negative_sector': negative_sector
        }

# Custom Collator for Triplets
class TripletCollator:
    def __call__(self, batch):
        anchor_summary = {
            'input_ids': torch.stack([item['anchor_summary']['input_ids'] for item in batch]),
            'attention_mask': torch.stack([item['anchor_summary']['attention_mask'] for item in batch])
        }
        positive_time_series = torch.stack([item['positive_time_series'] for item in batch])
        positive_sector = torch.stack([item['positive_sector'] for item in batch])
        negative_time_series = torch.stack([item['negative_time_series'] for item in batch])
        negative_sector = torch.stack([item['negative_sector'] for item in batch])

        return {
            'anchor_summary': anchor_summary,
            'positive_time_series': positive_time_series,
            'positive_sector': positive_sector,
            'negative_time_series': negative_time_series,
            'negative_sector': negative_sector
        }


def train_single_split(json_file, train_val_split=0.8):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    dataset = TimeSeriesPolicyTripletDataset(json_file, tokenizer)

    # Split dataset into train and validation set (80% train, 20% val by default)
    train_size = int(train_val_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_dataloader = DataLoader(train_dataset, batch_size=64, collate_fn=TripletCollator(), shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=64, collate_fn=TripletCollator())

    # Initialize the model
    model = TimeSeriesPolicyEmbeddingModel(model_name=model_name, lr=1e-4)

    # Initialize CSVLogger
    csv_logger = CSVLogger(save_dir=log_dir)

    # Early stopping callback
    early_stop_callback = pl.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,  # Stop training after 5 epochs with no improvement
        mode="min"
    )

    # Model checkpoint callback
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor="val_loss",
        dirpath=csv_logger.log_dir,  # Save checkpoints in the same log directory
        filename="best-checkpoint",
        save_top_k=1,
        mode="min"
    )

    # Trainer with CSVLogger
    trainer = pl.Trainer(
        max_epochs=50,
        callbacks=[early_stop_callback, checkpoint_callback],
        logger=csv_logger  # Use CSVLogger
    )

    # Train the model on the single train/val split
    trainer.fit(model, train_dataloader, val_dataloader)


# Example usage for training with a single train/val split
model_name = 'climatebert/distilroberta-base-climate-f'
data_file = os.path.join(data_dir, 'train_20.jsonl')
# train_single_split(data_file, train_val_split=0.8)

# Inference

In [6]:
import torch
import pandas as pd
import json
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

# Load a single model from the checkpoint
def load_model(model_class, checkpoint_path, model_name, device='cuda'):
    model = model_class.load_from_checkpoint(checkpoint_path)
    model.eval()  # Set to evaluation mode
    model.to(device)  # Move model to GPU if available
    return model

# Load tokenizer
def load_tokenizer(model_name):
    return AutoTokenizer.from_pretrained(model_name)

# Embed the summary using the model
def embed_summary(summary, tokenizer, model, device='cuda'):
    inputs = tokenizer(summary, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get embeddings from the model
    with torch.no_grad():
        summary_embedding = model.get_text_embedding(inputs['input_ids'], inputs['attention_mask'])

    return summary_embedding.cpu().numpy()

# Embed time series + sector using the model
def embed_ts_sector(time_series, sector, model, device='cuda'):
    time_series = torch.tensor(time_series, dtype=torch.float32).unsqueeze(0).to(device)
    sector = torch.tensor(sector, dtype=torch.float32).unsqueeze(0).to(device)

    # Get embeddings from the model
    with torch.no_grad():
        ts_sector_embedding = model.get_ts_sector_embedding(time_series, sector)

    return ts_sector_embedding.cpu().numpy()

# General function to calculate hit@5 score for each filter type
def calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type=None, device='cuda'):
    hits = 0
    total = len(test_data)

    for test_sample in test_data:
        test_ts_embedding = embed_ts_sector(test_sample['positive_time_series'], test_sample['positive_sector'], model, device)
        # Apply the relevant filter
        if filter_type == 'sector':
            filtered_documents_df = documents_df[documents_df['Sector'] == ';'.join(test_sample['sector'])]
        elif filter_type == 'country':
            filtered_documents_df = documents_df[documents_df['Geography'] == test_sample['country']]
        elif filter_type == 'sector_and_country':
            filtered_documents_df = documents_df[
                (documents_df['Sector'] == ';'.join(test_sample['sector'])) &
                (documents_df['Geography'] == test_sample['country'])
            ]
        else:
            filtered_documents_df = documents_df  # No filtering

        if len(filtered_documents_df) == 0:
            continue  # Skip if no matching documents after filtering

        # Compute cosine similarity
        similarities = cosine_similarity(test_ts_embedding, np.vstack(filtered_documents_df['embedding']))

        # Get top 5 documents
        top_5_indices = similarities.argsort()[0][-5:]
        top_5_docs = filtered_documents_df.iloc[top_5_indices]['Document ID'].values

        # Check if the document ID from the test sample is in the top 5 retrieved docs
        if test_sample['doc_id'] in top_5_docs:
            hits += 1

    # Calculate hit@5 score
    hit_at_5_score = (hits / total)
    return hit_at_5_score

# Load document embeddings and the test data, and embed summaries using the model
def load_documents_and_test_data(documents_file, test_file, tokenizer, model, device='cuda'):
    # Load document database and compute embeddings for summaries
    documents_df = pd.read_csv(documents_file)
    documents_df['embedding'] = documents_df['Family Summary'].apply(lambda x: embed_summary(x, tokenizer, model, device))

    # Load test data
    with open(test_file, 'r') as f:
        test_data = json.load(f)

    return documents_df, test_data

# Example usage
model_name = 'climatebert/distilroberta-base-climate-f'
# model_name = 'bert-base-uncased'
version = 'mlp-climatebert-1-20'

checkpoint_path = os.path.join(log_dir, 'lightning_logs', f'{version}/best-checkpoint.ckpt')  # Adjust path as needed
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model and tokenizer
model = load_model(TimeSeriesPolicyEmbeddingModel, checkpoint_path, model_name, device)
tokenizer = load_tokenizer(model_name)

# Load documents and test data
documents_df, test_data = load_documents_and_test_data('all_data_en.csv', 'test.json', tokenizer, model, device)

# Calculate hit@5 scores for each filtering scenario
hit_at_5_no_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type=None, device=device)
print(f'Hit@5 Score (No Filter): {hit_at_5_no_filter:.2f}')

hit_at_5_sector_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type='sector', device=device)
print(f'Hit@5 Score (Sector Filter): {hit_at_5_sector_filter:.2f}')

hit_at_5_country_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type='country', device=device)
print(f'Hit@5 Score (Country Filter): {hit_at_5_country_filter:.2f}')

hit_at_5_sector_and_country_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type='sector_and_country', device=device)
print(f'Hit@5 Score (Sector + Country Filter): {hit_at_5_sector_and_country_filter:.2f}')

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# import torch
# import pandas as pd
# import json
# from transformers import AutoTokenizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# import os
# import pickle
# from tqdm.notebook import tqdm

# # Load a single model from the checkpoint
# def load_model(model_class, checkpoint_path, model_name, device='cuda'):
#     model = model_class.load_from_checkpoint(checkpoint_path)
#     model.eval()  # Set to evaluation mode
#     model.to(device)  # Move model to GPU if available
#     return model

# # Load tokenizer
# def load_tokenizer(model_name):
#     return AutoTokenizer.from_pretrained(model_name)

# # Embed the summary using the model
# def embed_summary(summary, tokenizer, model, device='cuda'):
#     inputs = tokenizer(summary, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     # Get embeddings from the model
#     with torch.no_grad():
#         summary_embedding = model.get_text_embedding(inputs['input_ids'], inputs['attention_mask'])

#     return summary_embedding.cpu().numpy()

# # General function to calculate hit@5 score for each filter type
# def calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type=None, device='cuda'):
#     hits = 0
#     total = len(test_data)

#     for test_sample in test_data:
#         test_ts_embedding = embed_ts_sector(test_sample['positive_time_series'], test_sample['positive_sector'], model, device)

#         # Apply the relevant filter
#         if filter_type == 'sector':
#             filtered_documents_df = documents_df[documents_df['sector'] == ';'.join(test_sample['sector'])]
#         elif filter_type == 'country':
#             filtered_documents_df = documents_df[documents_df['geography'] == test_sample['country']]
#         elif filter_type == 'sector_and_country':
#             filtered_documents_df = documents_df[
#                 (documents_df['sector'] == ';'.join(test_sample['sector'])) &
#                 (documents_df['geography'] == test_sample['country'])
#             ]
#         else:
#             filtered_documents_df = documents_df  # No filtering

#         if len(filtered_documents_df) == 0:
#             continue  # Skip if no matching documents after filtering

#         # Compute cosine similarity
#         similarities = cosine_similarity(test_ts_embedding, np.vstack(filtered_documents_df['embedding']))

#         # Get top 5 unique documents by document_id
#         top_5_docs = []
#         seen_doc_ids = set()
#         for index in similarities.argsort()[0][::-1]:  # Sort in descending order
#             doc_id = filtered_documents_df.iloc[index]['document_id']
#             if doc_id not in seen_doc_ids:
#                 top_5_docs.append(doc_id)
#                 seen_doc_ids.add(doc_id)
#             if len(top_5_docs) == 5:  # Stop once we have 5 unique document IDs
#                 break

#         # Check if the document ID from the test sample is in the top 5 retrieved docs
#         if test_sample['doc_id'] in top_5_docs:
#             hits += 1

#     # Calculate hit@5 score
#     hit_at_5_score = (hits / total)
#     return hit_at_5_score

# # Load chunks and test data, and embed summaries using the model
# def load_chunks_and_test_data(chunks_file, test_file, tokenizer, model, device='cuda'):
#     # Load chunks from pickle file
#     with open(chunks_file, 'rb') as f:
#         chunks = pickle.load(f)

#     # Convert chunks to DataFrame and compute embeddings for summaries
#     documents_df = pd.DataFrame([{
#         'summary': chunk.page_content,
#         'embedding': embed_summary(chunk.page_content, tokenizer, model, device),
#         'document_id': chunk.metadata['document_id'],
#         'sector': chunk.metadata['sector'],
#         'geography': chunk.metadata['geography']
#     } for chunk in tqdm(chunks)])

#     # Load test data
#     with open(test_file, 'r') as f:
#         test_data = json.load(f)

#     return documents_df, test_data

# # Example usage
# model_name = 'climatebert/distilroberta-base-climate-f'
# version = 'mlp-climatebert-1-20'
# checkpoint_path = os.path.join(log_dir, 'lightning_logs', f'{version}/best-checkpoint.ckpt')  # Adjust path as needed
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # Load the model and tokenizer
# model = load_model(TimeSeriesPolicyEmbeddingModel, checkpoint_path, model_name, device)
# tokenizer = load_tokenizer(model_name)

# # Load documents and test data
# documents_df, test_data = load_chunks_and_test_data('chunks.pkl', 'test.json', tokenizer, model, device)

# # # Calculate hit@5 scores for each filtering scenario
# # hit_at_5_no_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type=None, device=device)
# # print(f'Hit@5 Score (No Filter): {hit_at_5_no_filter:.2f}')

# # hit_at_5_sector_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type='sector', device=device)
# # print(f'Hit@5 Score (Sector Filter): {hit_at_5_sector_filter:.2f}')

# # hit_at_5_country_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type='country', device=device)
# # print(f'Hit@5 Score (Country Filter): {hit_at_5_country_filter:.2f}')

# # hit_at_5_sector_and_country_filter = calculate_hit_at_5_by_filter_type(test_data, documents_df, model, filter_type='sector_and_country', device=device)
# # print(f'Hit@5 Score (Sector + Country Filter): {hit_at_5_sector_and_country_filter:.2f}')

Some weights of RobertaModel were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/148108 [00:00<?, ?it/s]

Hit@5 Score (No Filter): 0.04
Hit@5 Score (Sector Filter): 0.54
Hit@5 Score (Country Filter): 0.90
Hit@5 Score (Sector + Country Filter): 0.98


## Error Analysis

In [12]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # General function to calculate hit@5 score for each filter type and return correct/incorrect docs
# def calculate_hit_at_5_with_error_analysis(test_data, documents_df, model, device='cuda'):
#     hits = 0
#     total = len(test_data)
#     correct_docs = []
#     incorrect_docs = []

#     for test_sample in test_data:
#         test_ts_embedding = embed_ts_sector(test_sample['positive_time_series'], test_sample['positive_sector'], model, device)

#         # Compute cosine similarity
#         similarities = cosine_similarity(test_ts_embedding, np.vstack(documents_df['embedding']))

#         # Get top 5 unique documents by document_id
#         top_5_docs = []
#         seen_doc_ids = set()
#         for index in similarities.argsort()[0][::-1]:  # Sort in descending order
#             doc_id = documents_df.iloc[index]['Document ID']
#             if doc_id not in seen_doc_ids:
#                 top_5_docs.append(doc_id)
#                 seen_doc_ids.add(doc_id)
#             if len(top_5_docs) == 5:  # Stop once we have 5 unique document IDs
#                 break

#         print(test_sample)

#         # Check if the document ID from the test sample is in the top 5 retrieved docs
#         if test_sample['doc_id'] in top_5_docs:
#             hits += 1
#             correct_docs.append({
#                 'doc_id': test_sample['doc_id'],
#                 'similarity': similarities[0].max(),  # Maximum similarity score for correct doc
#                 'sector': ';'.join(test_sample['sector']),
#                 'geography': test_sample['country'],
#                 'summary': test_sample['anchor_summary']
#             })
#         else:
#             incorrect_docs.append({
#                 'doc_id': test_sample['doc_id'],
#                 'similarity': similarities[0].max(),  # Maximum similarity score for the top doc
#                 'sector': ';'.join(test_sample['sector']),
#                 'geography': test_sample['country'],
#                 'summary': test_sample['anchor_summary']
#             })

#     # Calculate hit@5 score
#     hit_at_5_score = (hits / total)
#     return hit_at_5_score, correct_docs, incorrect_docs

# # Save correct and incorrect predictions to CSV
# def save_correct_incorrect_docs(correct_docs, incorrect_docs, correct_csv='correct_docs.csv', incorrect_csv='incorrect_docs.csv'):
#     correct_df = pd.DataFrame(correct_docs)
#     incorrect_df = pd.DataFrame(incorrect_docs)

#     correct_df.to_csv(correct_csv, index=False)
#     incorrect_df.to_csv(incorrect_csv, index=False)

# # Plot cosine similarity score distributions for correct and incorrect predictions
# def plot_cosine_similarity_distributions(correct_docs, incorrect_docs):
#     correct_similarities = [doc['similarity'] for doc in correct_docs]
#     incorrect_similarities = [doc['similarity'] for doc in incorrect_docs]

#     plt.figure(figsize=(10, 6))
#     sns.histplot(correct_similarities, bins=20, color='green', label='Correct', kde=True)
#     sns.histplot(incorrect_similarities, bins=20, color='red', label='Incorrect', kde=True)
#     plt.xlabel('Cosine Similarity Score')
#     plt.ylabel('Frequency')
#     plt.title('RTCPR Siamese Network Model: Cosine Similarity Distribution: Correct vs Incorrect Predictions')
#     plt.legend()
#     plt.savefig('rtcpr-cosine.svg', format='svg')
#     plt.show()

# # Example usage
# hit_at_5_no_filter, correct_docs, incorrect_docs = calculate_hit_at_5_with_error_analysis(test_data, documents_df, model, device=device)
# print(f'Hit@5 Score (No Filter): {hit_at_5_no_filter:.2f}')

# # Save correct and incorrect docs to CSV files
# save_correct_incorrect_docs(correct_docs, incorrect_docs, correct_csv='correct_docs_no_filter.csv', incorrect_csv='incorrect_docs_no_filter.csv')

# # Plot cosine similarity distributions
# plot_cosine_similarity_distributions(correct_docs, incorrect_docs)

Output hidden; open in https://colab.research.google.com to view.